In [ ]:
from os.path import join
from glob import glob
import numpy as np
import dask
import dask.array as da
import pyklb as klb
import napari
import time

In [ ]:
#Use opportunistic caching 
from dask.cache import Cache
cache = Cache(8e9)  # Leverage some memory
cache.register()    # Turn cache on globally

In [ ]:
inPath = 'W:/SV3/RC_15-06-11/Dme_E2_His2AvRFP_spiderGFP_12-03_20150611_155054.corrected/Results/TimeFused/'
fnames = glob(join(inPath, '*/*CM00_CHN00.fusedStack.klb' ))
fnames.sort()

sample_full = klb.readfull(fnames[0]) #Sample image with klb.readfull
z= sample_full.shape[0]
y= sample_full.shape[1]
x= sample_full.shape[2]

In [ ]:
# Segment each XY-image in the 3D stack is into a 2x2 grid

# X and Y strides to segment the each XY-image
y_stride= y/2
x_stride= x/2

ROIs_lobo = ([0,0,0],
             [0,y_stride,0],
             
             [0,0,x_stride],
             [0,y_stride,x_stride]) 

ROIs_upbo = ([z-1,y_stride-1,x_stride-1],
             [z-1,2*y_stride-1,x_stride-1],
             
             [z-1,y_stride-1,2*x_stride-1],
             [z-1,2*y_stride-1,2*x_stride-1])

sample = klb.readroi(fnames[0], ROIs_lobo[0], ROIs_upbo[0]) #Sample image with klb.readroi
print(sample.shape, sample.dtype)

### Generate lazy arrays:

In [ ]:
# For each ROI, lazily evaluate klb.readroi with dask.delayed 
lazy_values = [[dask.delayed(klb.readroi)(fn,ROIs_lobo[i], ROIs_upbo[i]) for fn in fnames] for i in range(len(ROIs_lobo))]

In [ ]:
#Construct lazy dask arrays for all lazy values
start = time.time()
lazy_arrays_0 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[0]]

lazy_arrays_1 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[1]]

lazy_arrays_2 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[2]]

lazy_arrays_3 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[3]]

#Concatenate the ROIs
foo = da.concatenate([lazy_arrays_0, lazy_arrays_1], axis = 2)
foos = da.concatenate([lazy_arrays_2, lazy_arrays_3], axis = 2)
a = da.concatenate([foo, foos], axis = 3)

print(type(a), a.shape, a.dtype)
print('Image-data on disk:', 2*round(a.size/(1024**3),2), 'GB (uncompressed)')
print('Chunksize:', a.chunksize)
print('Took', round(time.time()-start,2), 'sec')

###  Initialize the GUI first

In [ ]:
%gui qt5

### Create a napari viewer and add the image as a layer

In [ ]:
import napari
viewer = napari.Viewer(axis_labels = ['t','z','y','x'])
viewer.add_image(a,
                 scale = [1,8,1,1],
                 colormap='inferno',
                 blending='additive',
                 name='membrane',
                 is_pyramid = False,
                 rgb = False,
                 contrast_limits = [10,255])

#### Let's grab the 2nd color channel image-stacks and add it to the viewer

In [ ]:
fnames = glob(join(inPath, '*/*CM00_CHN01.fusedStack.klb' ))
fnames.sort()

# For each ROI, lazily evaluate klb.readroi with dask.delayed 
lazy_values = [[dask.delayed(klb.readroi)(fn,ROIs_lobo[i], ROIs_upbo[i]) for fn in fnames] for i in range(len(ROIs_lobo))]

#Construct lazy dask arrays for all lazy values
lazy_arrays_0 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[0]]

lazy_arrays_1 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[1]]

lazy_arrays_2 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[2]]

lazy_arrays_3 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[3]]

#Concatenate the ROIs
foo = da.concatenate([lazy_arrays_0, lazy_arrays_1], axis = 2)
foos = da.concatenate([lazy_arrays_2, lazy_arrays_3], axis = 2)
b = da.concatenate([foo, foos], axis = 3)

viewer.add_image(b,
                 scale = [1,8,1,1],
                 colormap='green',
                 blending='additive',
                 name='nuclei',
                 is_pyramid = False,
                 rgb = False,
                 contrast_limits = [10,355])

#### Let's grab the orthogonal views and add those to the viewer too

In [ ]:
fnames = glob(join(inPath, '*/*CM01_CHN00.fusedStack.klb' ))
fnames.sort()

# For each ROI, lazily evaluate klb.readroi with dask.delayed 
lazy_values = [[dask.delayed(klb.readroi)(fn,ROIs_lobo[i], ROIs_upbo[i]) for fn in fnames] for i in range(len(ROIs_lobo))]

#Construct lazy dask arrays for all lazy values
lazy_arrays_0 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[0]]

lazy_arrays_1 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[1]]

lazy_arrays_2 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[2]]

lazy_arrays_3 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[3]]

#Concatenate the ROIs
foo = da.concatenate([lazy_arrays_0, lazy_arrays_1], axis = 2)
foos = da.concatenate([lazy_arrays_2, lazy_arrays_3], axis = 2)
c = da.concatenate([foo, foos], axis = 3)

viewer.add_image(c,
                 scale = [1,8,1,1],
                 colormap='inferno',
                 blending='additive',
                 name='membrane, side-view',
                 is_pyramid = False,
                 rgb = False,
                 contrast_limits = [10,255],
                translate=[725,180])

In [ ]:
fnames = glob(join(inPath, '*/*CM01_CHN01.fusedStack.klb' ))
fnames.sort()

# For each ROI, lazily evaluate klb.readroi with dask.delayed 
lazy_values = [[dask.delayed(klb.readroi)(fn,ROIs_lobo[i], ROIs_upbo[i]) for fn in fnames] for i in range(len(ROIs_lobo))]

#Construct lazy dask arrays for all lazy values
lazy_arrays_0 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[0]]

lazy_arrays_1 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[1]]

lazy_arrays_2 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[2]]

lazy_arrays_3 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[3]]

#Concatenate the ROIs
foo = da.concatenate([lazy_arrays_0, lazy_arrays_1], axis = 2)
foos = da.concatenate([lazy_arrays_2, lazy_arrays_3], axis = 2)
d = da.concatenate([foo, foos], axis = 3)

viewer.add_image(d,
                 scale = [1,8,1,1],
                 colormap='green',
                 blending='additive',
                 name='nuclei, side-view',
                 is_pyramid = False,
                 rgb = False,
                 contrast_limits = [10,355],
                translate=[725,180])